In [ ]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
#from sklearn.preprocessing import PolynomialFeatures
from multiprocessing import Pool
import multiprocessing
import ZVnbrosse
from potentials import PotentialOne, PotentialTwo, PotentialThree, PotentialOnePrime
from zv_cv import Eval_ZVCV_Gaus
from samplers import MCMC_sampler,Generate_train,ULA_light
from baselines import set_function,construct_ESVM_kernel,GenerateSigma, construct_Tukey_Hanning, set_function, Spectral_var, mult_W 
from martingale import approx_q,test_traj
from optimize import Run_eval_test,optimize_parallel_new 
from utils import *
import copy

In [ ]:
N_burn = 1*10**3 # Burn in period
N_train = 1*10**4 # Number of samples on which we optimize
step = 0.05 # Step size
#put 0.5 for MALA
#step = 0.2
n_traj = 100 # Number of independent MCMC trajectories for test
f_type = "sum"
bn = int(np.sqrt(N_train))
W_test = construct_Tukey_Hanning(N_train,bn)

Choose density parameters

In [ ]:
d = 2
#B = np.pi/2
M = 1.0
mu = 3.0
sigma = 3.0
Cur_pot = PotentialThree(M,mu,sigma,d)
#mu = 0.5
#sigma = 0.5
#M = 1.0
#Cur_pot = PotentialTwo(M,mu,sigma,d)

### Visualize level sets of the potential

In [ ]:
mu_1 = 3.0
mu_2 = 3.0
M = 1.0
sigma = 3.0
###
def U(x, y):
    return -1./(2*M**2)*(np.sqrt(x**2 + y**2) - mu_1)**2 + np.log(np.exp(-1/(2*sigma**2)*(x-mu_2)**2) + np.exp(-1/(2*sigma**2)*(x+mu_2)**2))

x = np.linspace(-5, 5, 200)
y = np.linspace(-5, 5, 200)

X, Y = np.meshgrid(x, y)
Z = np.exp(U(X, Y))

In [ ]:
plt.imshow(Z, extent=[-5, 5, -5, 5], origin='lower',
           cmap='RdGy')
plt.colorbar()
plt.savefig("rezende_density_profile.pdf")
plt.axis(aspect='image')

In [ ]:
M = 10
B = 0.1
def U(x,y):
    return -1./(2*M)*x**2 - 1./2*(y+B*x**2-M*B)**2

x = np.linspace(-5, 5, 200)
y = np.linspace(-5, 5, 200)

X, Y = np.meshgrid(x, y)
Z = np.exp(U(X, Y))

In [ ]:
plt.imshow(Z, extent=[-5, 5, -5, 5], origin='lower',
           cmap='RdGy')
plt.colorbar()
plt.savefig("banana_shape.pdf")
plt.axis(aspect='image')

### Generate data

In [ ]:
sampler = {"sampler":"ULA","burn_type":"full","main_type":"full"} # Sampling method

if sampler["sampler"] == "ULA":
    res = Generate_train(n_traj, sampler, Cur_pot, step, N_burn, N_train, d)
    res = np.asarray(res)
    traj,traj_grad = res[:,0,:,:],res[:,1,:,:]
else:
    res = Generate_train(n_traj, sampler, Cur_pot, step, N_burn, N_train, d)
    traj = []
    traj_grad = []
    for i in range(len(res)):
        traj.append(res[i][0])
        traj_grad.append(res[i][1])
        print("accepted = ",res[i][2])
    traj = np.asarray(traj)
    traj_grad = np.asarray(traj_grad)

### Visualize training trajectories

In [ ]:
visualize_scatter_2d(traj[0,:,:])

### Run variance reduction

In [ ]:
print(traj.shape)
print(traj_grad.shape)
traj_grad = (-1)*traj_grad

In [ ]:
from zv_cv import ZVpolyOne
def GausCV(traj,sample):
    """
    returns matrix of gaussian CV's 
    """
    m=7
    pen=0.
    x = np.linspace(-3,3,m)
    y = np.linspace(-3,3,m)
    sigma_squared = 4.0
    xx, yy = np.meshgrid(x,y)
    d = m**2
    mu = np.concatenate((xx.reshape((-1,1)),yy.reshape((-1,1))),axis=1)
    traj_adj = (np.repeat(traj[:,np.newaxis,:], d, axis=1)-mu[np.newaxis,:])/sigma_squared
    psi_matr = np.zeros((d,traj.shape[0]))
    for i in range(d):
        psi_matr[i,:] = np.exp(-np.sum((traj-mu[i].reshape((1,2)))**2, axis=1)/(2*sigma_squared))
    cov = np.dot(psi_matr - psi_matr.mean(axis=0),sample - sample.mean()) / traj.shape[0]
    jac_matr = -traj_adj*((psi_matr.T).reshape((psi_matr.shape[1],psi_matr.shape[0],1)))
    H = np.mean(np.matmul(jac_matr,jac_matr.transpose(0,2,1)),axis=0)
    param_CV = np.linalg.inv(H + pen*np.eye(H.shape[0])) @ cov                                                                                 
    jac_star = np.sum(jac_matr*param_CV[np.newaxis,:],axis=1)
    print(jac_star.shape)
    delta_star = (psi_matr.T*(np.sum(traj_adj**2, axis=2)-traj.shape[1]/sigma_squared)).dot(param_CV)
    return jac_star,delta_star

def GausCV_adj(traj,traj_grad,samples,f_target,params,W_spec):
    """
    returns matrix of gaussian CV's 
    """
    #m=7 - good
    m=7
    lambda_reg=0.
    x = np.linspace(-5,5,m)
    y = np.linspace(-5,5,m)
    sigma = 2.0
    xx, yy = np.meshgrid(x,y)
    d = m**2
    #print(xx)
    mu = np.concatenate((xx.reshape((-1,1)),yy.reshape((-1,1))),axis=1)
    #Nabla_psi = np.zeros((d,samples.shape[0]),dtype=float)
    Psi = np.zeros((d,samples.shape[0]),dtype=float)
    L_psi = np.zeros((d,samples.shape[0]),dtype=float)
    for i in range(d):
        #Nabla_psi[i] = (i+1)*(traj[:,0]**i)
        Psi[i] = np.exp(-(np.sum((traj-mu[i])**2,axis=1)/(2*sigma**2)))
        L_psi[i] = (np.sum(traj_grad*(traj-mu[i])/sigma**2,axis=1) \
                    +(np.sum((traj-mu[i])**2,axis=1)/sigma**4 - traj.shape[1]/sigma**2))*Psi[i]
    #compute main matrix
    Pois = np.concatenate([Psi,-L_psi],axis=0)
    Cov_matr = np.cov(Pois,rowvar = True)
    H_cv = Cov_matr[:d,d:]
    b_cv = ((Psi - Psi.mean(axis=1).reshape(d,1)) @ (samples - samples.mean(axis=0)))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_cv + lambda_reg*np.eye(d)) @ b_cv
    CV_est = samples + L_psi.T @ theta
    mean_CV = np.mean(CV_est, axis = 0)
    var_CV = Spectral_var(CV_est[:,0],W_spec)
    return mean_CV, var_CV 

def GausZV(traj,traj_grad,samples,f_target,params,W_spec):
    """
    returns matrix of gaussian CV's for ZV algorithm;
    """
    #m=7 - good
    m=7
    lambda_reg=0.
    x = np.linspace(-5,5,m)
    y = np.linspace(-5,5,m)
    sigma = 2.0
    xx, yy = np.meshgrid(x,y)
    d = m**2
    #print(xx)
    mu = np.concatenate((xx.reshape((-1,1)),yy.reshape((-1,1))),axis=1)
    #Nabla_psi = np.zeros((d,samples.shape[0]),dtype=float)
    Psi = np.zeros((d,samples.shape[0]),dtype=float)
    L_psi = np.zeros((d,samples.shape[0]),dtype=float)
    for i in range(d):
        #Nabla_psi[i] = (i+1)*(traj[:,0]**i)
        Psi[i] = np.exp(-(np.sum((traj-mu[i])**2,axis=1)/(2*sigma**2)))
        L_psi[i] = (np.sum(traj_grad*(traj-mu[i])/sigma**2,axis=1) \
                    +(np.sum((traj-mu[i])**2,axis=1)/sigma**4 - traj.shape[1]/sigma**2))*Psi[i]
    #compute main matrix
    H_zv = np.cov(L_psi,rowvar = True)
    b_zv = ((L_psi - L_psi.mean(axis=1).reshape(d,1)) @ (samples - samples.mean(axis=0)))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_zv + lambda_reg*np.eye(d)) @ b_zv
    ZV_est = samples - L_psi.T @ theta
    mean_ZV = np.mean(ZV_est, axis = 0)
    var_ZV = Spectral_var(ZV_est[:,0],W_spec)
    return mean_ZV, var_ZV 

def GausESVM(traj,traj_grad,samples,f_target,params,W_spec):
    """
    returns matrix of gaussian CV's for ESVM argorithm;
    """
    #m=7 - good
    m=7
    lambda_reg=0.
    x = np.linspace(-5,5,m)
    y = np.linspace(-5,5,m)
    sigma = 2.0
    xx, yy = np.meshgrid(x,y)
    d = m**2
    #print(xx)
    mu = np.concatenate((xx.reshape((-1,1)),yy.reshape((-1,1))),axis=1)
    #Nabla_psi = np.zeros((d,samples.shape[0]),dtype=float)
    Psi = np.zeros((d,samples.shape[0]),dtype=float)
    L_psi = np.zeros((d,samples.shape[0]),dtype=float)
    for i in range(d):
        #Nabla_psi[i] = (i+1)*(traj[:,0]**i)
        Psi[i] = np.exp(-(np.sum((traj-mu[i])**2,axis=1)/(2*sigma**2)))
        L_psi[i] = (np.sum(traj_grad*(traj-mu[i])/sigma**2,axis=1) \
                    +(np.sum((traj-mu[i])**2,axis=1)/sigma**4 - traj.shape[1]/sigma**2))*Psi[i]
    #compute main matrix
    H_esvm = np.zeros((d,d),dtype = float)
    for i in range(d):
        for j in range(d):
            H_esvm[i,j] = np.dot(L_psi[i] - np.mean(L_psi[i]), mult_W(L_psi[j] - np.mean(L_psi[j]),W_spec))/(samples.shape[0]-1)
    #compute right-hand side
    b_esvm = np.zeros(d, dtype = float)
    for i in range(d):
        b_esvm[i] = np.dot(L_psi[i] - np.mean(L_psi[i]), mult_W(samples - samples.mean(axis=0),W_spec))/(samples.shape[0]-1)
    theta = np.linalg.inv(H_esvm + lambda_reg*np.eye(d)) @ b_esvm
    ESVM_est = samples - L_psi.T @ theta
    mean_ESVM = np.mean(ESVM_est, axis = 0)
    var_ESVM = Spectral_var(ESVM_est[:,0],W_spec)
    return mean_ESVM, var_ESVM    

def Eval_ZVCV_Gaus(traj,traj_grad, f_target, params, W_spec):
    if f_target == "sum":
        samples = traj.sum(axis = 1).reshape(-1,1)
    elif f_target == "sum_comps":
        samples = traj[:,params["ind"]].reshape(-1,1)
    elif f_target == "sum_comps_squared":
        samples = np.square(traj[:,params["ind"]]).reshape(-1,1)
    elif f_target == "sum_squared":
        samples = np.square(traj).sum(axis = 1).reshape(-1,1)
    elif f_target == "sum_4th":
        samples = ((traj)**4).sum(axis = 1).reshape(-1,1)
    elif f_target == "exp_sum":
        samples = np.exp(traj.sum(axis = 1)).reshape(-1,1)
    else:
        traj = np.expand_dims(traj, axis = 0)
        samples = set_function(f_target,traj,[0],params)
        traj = traj[0]
        samples = samples[0]
    mean_vanilla = np.mean(samples)
    vars_vanilla = Spectral_var(samples[:,0],W_spec)
    mean_ZV, var_ZV = GausZV(traj,traj_grad,samples,f_target,params,W_spec)
    mean_CV_adj, var_CV_adj = GausCV_adj(traj,traj_grad,samples,f_target,params,W_spec) 
    mean_ESVM, var_ESVM = GausESVM(traj,traj_grad,samples,f_target,params,W_spec)
    return (mean_vanilla,mean_ZV,mean_CV_adj,mean_ESVM), (vars_vanilla,var_ZV,var_CV_adj,var_ESVM)

In [ ]:
test_seed = 1453
f_type = "sum"
params = {"ind":1}
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV_Gaus, [(traj[i,:,:],traj_grad[i,:,:],f_type,params,W_test) for i in range (n_traj)])
trav.close()

In [ ]:
res_arr = np.asarray(res)
print(res_arr.shape)

### Saving results

In [ ]:
#np.save("results/rezende/RWM_linear_29_06.npy",res_arr)
print("Average vr rates:")
print("ZV:",np.mean(res_arr[:,1,0]/res_arr[:,1,1]))
print("CV:",np.mean(res_arr[:,1,0]/res_arr[:,1,2]))
print("ESVM:",np.mean(res_arr[:,1,0]/res_arr[:,1,2]))

### Comparison plots

In [ ]:
title = ""
labels = ['Vanilla\n RWM', 'RWM \nwith ZV', 'RWM \nwith CV']

In [ ]:
data = [res_arr[:,0,0],res_arr[:,0,1],res_arr[:,0,2]] 
boxplot_ind(data, title, labels, path = "results/rezende/rwm.pdf")

In [ ]:
title = ""
labels = ['ULA \nwith ZV-1', 'ULA \nwith CV-1']

In [ ]:
data = [res_arr[:,0,1],res_arr[:,0,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr[:,0,0],res_arr[:,0,2],res_arr[:,0,4]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['ULA \nwith ZV-2', 'ULA \nwith CV-2']

In [ ]:
data = [res_arr[:,0,2],res_arr[:,0,4]] 
boxplot_ind(data, title, labels)